<a href="https://colab.research.google.com/github/Andres8bit/Machine-Learning/blob/main/vidoe_restoration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2

video = cv2.VideoCapture('/content/drive/MyDrive/Video_SuperImg/nasa_historic_photage.mp4')

In [ ]:
def vid_to_frame(video,sec,count):
  video.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
  hasFrames,image = video.read()
  if hasFrames:
    cv2.imwrite("/content/drive/MyDrive/Video_SuperImg/frames"+str(count)+".jpg",image)
  
  return hasFrames


In [ ]:
sec = 0
frameRate = 0.5
count = 1
success = vid_to_frame(video,sec,count)
while success:
  count = count + 1
  sec = sec + frameRate
  sec = round(sec,2)
  success = vid_to_frame(video,sec,count)

KeyboardInterrupt: ignored

In [4]:
from tensorflow import keras
import tensorflow as tf
model = keras.models.load_model("/content/drive/MyDrive/saved_models/super_img")
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 64)    1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 32)    18464     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 9)     2601      
_________________________________________________________________
tf.nn.depth_to_space (TFOpLa (None, None, None, 1)     0         
Total params: 59,657
Trainable params: 59,657
Non-trainable params: 0
_________________________________________________________

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory


img_dir = '/content/drive/MyDrive/Video_SuperImg/frames'
images =  [os.path.join(img_dir, x) for x in os.listdir(img_dir) if x.endswith('.jpg')]
print(os.listdir(img_dir))

[]


In [ ]:
def parse_images(path):
  content = tf.io.read_file(path)
  images = tf.image.decode_jpeg(content,channels=1)
  images = tf.image.resize(images,(512,512,3))
  return images

In [1]:
def load_dataset(dir):
  i = 0
  files = os.listdir(dir)
  data_set = np.zeros((len(files),240,320,3))
  for f in files:
    if f.endswith('.jpg') and i != data_set.shape[0]-1:
      temp = tf.io.read_file(dir + '/' + f)
      data = tf.image.decode_jpeg(temp,channels=3)  
      i += 1
      data_set[i] = data
  data_set = tf.cast(data_set,tf.float32)
  return data_set

In [2]:
import numpy as np
import os
import tensorflow as tf
dataset = load_dataset('/content/drive/MyDrive/Video_SuperImg')
dataset

<tf.Tensor: shape=(2976, 240, 320, 3), dtype=float32, numpy=
array([[[[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        ...,

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.

In [3]:
dataset.shape

TensorShape([2976, 240, 320, 3])

In [7]:
img = model.predict(dataset[0])

In [14]:
img.shape

(240, 960, 9, 1)